In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from copy import deepcopy
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser

from tqdm import tqdm
from typing import Dict

/tmp/ipykernel_303215/3662406367.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Preprocessing

## Removing trivial unfeasible triplets

In [49]:
parser = Parser("data/testfiles/test5.txt")

In [50]:
info = parser.read()

1204it [00:00, 144279.94it/s]

In [51]:
dataset : Dict[int, pd.DataFrame ]= info['data']
p, K, M, N = info['p'], info['K'], info['M'], info['N']

In [6]:
new_dataset = deepcopy(dataset)

sum_min = 0
for n in range(N):

    p_min_n = new_dataset[n]['p_k,m,n'].min()

    sum_min += p_min_n

for n in tqdm(range(N)):
    p_min_n = new_dataset[n]['p_k,m,n'].min()

    for idx, row in new_dataset[n].iterrows():
        p_kmn = row['p_k,m,n']

        if p_kmn + sum_min - p_min_n > p:
            # print(p_kmn, sum_min-p_min_n , row['k'], row['m'], n )
            new_dataset[n].drop(index=idx, inplace=True)
        


 30%|███       | 12/40 [00:00<00:00, 116.98it/s]

100%|██████████| 40/40 [00:00<00:00, 124.73it/s]


In [7]:
new_size = 0
for v in new_dataset.values():
    new_size += len(v)

new_size

1954

## Removing IP Dominated triplets

In [54]:
new_dataset = {}
for n in tqdm(range(N)):
    if len(dataset[n]) == 0:
        continue

    sorted_by_power = dataset[n].sort_values(
        by = "p_k,m,n"
    )

    r_max = -np.inf
    p_last : float = None
    idx_last : int = None
    removed_last : bool = False
    for i, (idx, row) in enumerate(sorted_by_power.iterrows()):
        
        r_kmn = row['r_k,m,n']
        p_kmn = row['p_k,m,n']

        if r_max >= r_kmn:
            sorted_by_power = sorted_by_power.drop(index= idx)
            removed_last = True
            
        elif p_kmn == p_last:
            if not removed_last:
                sorted_by_power= sorted_by_power.drop(index = idx_last)

                # print("removing element already removed")
    
        if r_kmn > r_max:
            removed_last = False
            r_max = r_kmn

        p_last = p_kmn
        idx_last = idx

    new_dataset[n] = sorted_by_power


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:00<00:00, 53.85it/s]


In [56]:
new_size = 0
for v in new_dataset.values():
    new_size += len(v)

new_size

329

## Removing LP Dominated triplets

In [57]:
df = deepcopy(new_dataset)

In [61]:
lp_dominated=  []
for n in tqdm(range(N)):


    if len(df[n]) <= 2:
        continue
    idx1 = 0
    idx2 = 1
    idx3 = 2

    while idx3 < len(df[n]):

        l1 = df[n].iloc[idx1]
        r1 = l1['r_k,m,n']
        p1 = l1['p_k,m,n']

    
        l2 = df[n].iloc[idx2]
        r2 = l2['r_k,m,n']
        p2 = l2['p_k,m,n']

        l3 = df[n].iloc[idx3]
        r3 = l3['r_k,m,n']
        p3 = l3['p_k,m,n']

        if (r3-r2) * (p2-p1) >= (r2- r1) * (p3 - p2):
            lp_dominated.append([l2.k, l2.m, n])

            if idx1 == 0:
                idx2 = idx3
                idx3 +=1

            else:
                idx2 = idx1
                idx1 -= 1

        else:
            idx3+=1
            idx2+=1
            idx1+=1

  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:00<00:00, 559.32it/s]


In [62]:
new_size - len(lp_dominated) 

256

In [42]:
for (k,m,n) in lp_dominated:

    idx_drop = dataset[n].query(f"k == {k} & m == {m}").index
    dataset[n] = dataset[n].drop(index=idx_drop)


In [48]:
idx_drop = dataset[n].query(f"k == {k} & m == {m}").index
dataset[n].drop(index=idx_drop)

,k,m,n,"p_k,m,n","r_k,m,n"
0,0,0,39,49.0,50.0
1,0,1,39,141.0,58.0
2,0,2,39,428.0,78.0
3,0,3,39,443.0,98.0
4,1,0,39,47.0,4.0
5,1,1,39,210.0,14.0
6,1,2,39,448.0,69.0
7,1,3,39,448.0,80.0
8,2,0,39,283.0,12.0
9,2,1,39,393.0,23.0


In [14]:
for n in tqdm(range(N)):

    if len(df[n]) <= 2:
        continue
    l1 = df[n].iloc[0]
    r1 = l1['r_k,m,n']
    p1 = l1['p_k,m,n']

    df[n]['rp_ratio'] = (df[n]['r_k,m,n'] - r1)/(df[n]['p_k,m,n']  - p1) 

    # sorted_by_ratio : pd.DataFrame = df[n].sort_values(by = "rp_ratio")

    convex_hull = [pd.DataFrame(l1).T]
    for (idx, row) in df[n].iterrows():
        if idx == l1.name: 
            continue
        
        if len(convex_hull ) <= 1:
            convex_hull.append(pd.DataFrame(row).T)
            continue
        
        l_1  = convex_hull[-1]
        r_1 = l_1['r_k,m,n'].item()
        p_1 = l_1['p_k,m,n'].item()

        l_2 = convex_hull[-2]
        r_2 = l_2['r_k,m,n'].item()
        p_2 = l_2['p_k,m,n'].item()

        r = row['r_k,m,n'].item()
        p = row['p_k,m,n'].item()

        while (r_1 - r) * (p_2 - p_1) >= (r_2 - r_1) * (p_1 - p) and len(convex_hull) > 2:
            convex_hull.pop()

            l_1  = convex_hull[-1]
            r_1 = l_1['r_k,m,n'].item()
            p_1 = l_1['p_k,m,n'].item()

            l_2 = convex_hull[-2]
            r_2 = l_2['r_k,m,n'].item()
            p_2 = l_2['p_k,m,n'].item()

        
        convex_hull.append(pd.DataFrame(row).T)

    df[n] = pd.concat(convex_hull)

100%|██████████| 40/40 [00:00<00:00, 201.18it/s]


In [15]:
new_size = 0
for v in df.values():
    new_size += len(v)

new_size

212

In [407]:
df[1]

,k,m,n,"p_k,m,n","r_k,m,n"
4,2,0,1,1.0,50.0
5,2,1,1,7.0,85.0


In [268]:
df[n]['rp_ratio'] = (df[n]['r_k,m,n'] - r1)/(df[n]['p_k,m,n']  - p1) 

In [269]:
df[n]

,k,m,n,"p_k,m,n","r_k,m,n",rp_ratio
144,9,0,639,7.0,7.0,NaN
160,10,0,639,65.0,11.0,0.068966
720,45,0,639,125.0,18.0,0.093220
913,57,1,639,188.0,30.0,0.127072
914,57,2,639,513.0,39.0,0.063241
240,15,0,639,1114.0,42.0,0.031617
35,2,3,639,1230.0,47.0,0.032706
325,20,5,639,1499.0,58.0,0.034182
326,20,6,639,2326.0,66.0,0.025442
327,20,7,639,3345.0,68.0,0.018274
